In [2]:
import os
import tarfile
import urllib.request
import pandas as pd

In [3]:
# Chemins et URL
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dossier_data = "data"
fichier_tar = os.path.join(dossier_data, "aclImdb_v1.tar.gz")
dossier_extrait = os.path.join(dossier_data, "aclImdb")

# Créer le dossier data s'il n'existe pas
os.makedirs(dossier_data, exist_ok=True)

# Télécharger le fichier si ce n'est pas déjà fait
if not os.path.exists(fichier_tar):
    print("Téléchargement des données...")
    urllib.request.urlretrieve(url, filename=fichier_tar)
    print("Téléchargement terminé.")
else:
    print("Le fichier existe déjà.")

# Extraire les données si ce n'est pas déjà fait
if not os.path.exists(dossier_extrait):
    print("Extraction des données...")
    with tarfile.open(fichier_tar, "r:gz") as tar:
        tar.extractall(path=dossier_data)
    print("Extraction terminée.")
else:
    print("Les données sont déjà extraites.")

Téléchargement des données...
Téléchargement terminé.
Extraction des données...
Extraction terminée.


In [4]:
# Chemins
dossier_train = os.path.join("data", "aclImdb", "train")
chemins = {
    "pos": os.path.join(dossier_train, "pos"),
    "neg": os.path.join(dossier_train, "neg")
}

# Fonction pour charger les fichiers
def charger_avis(dossier, label):
    textes = []
    for nom_fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, nom_fichier)
        with open(chemin_fichier, encoding="utf-8") as f:
            textes.append(f.read())
    return pd.DataFrame({"texte": textes, "label": label})

# Charger les avis positifs et négatifs
df_pos = charger_avis(chemins["pos"], label=1)
df_neg = charger_avis(chemins["neg"], label=0)

# Combiner en un seul DataFrame
df_train = pd.concat([df_pos, df_neg], ignore_index=True)

# Mélanger les lignes
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

# Aperçu
df_train.head()

,texte,label
0,This is a great movie from the lost age of rea...,1
1,ALL FOR LOVE ( as it titled when it was broadc...,0
2,"The Unborn is a very, very different film. Jam...",1
3,"Wow, here it finally is; the action ""movie"" wi...",0
4,I agree totally with the last commenter this c...,0


In [5]:
# Chemins pour les données de test
dossier_test = os.path.join("data", "aclImdb", "test")
chemins_test = {
    "pos": os.path.join(dossier_test, "pos"),
    "neg": os.path.join(dossier_test, "neg")
}

# Charger les avis positifs et négatifs pour test
df_test_pos = charger_avis(chemins_test["pos"], label=1)
df_test_neg = charger_avis(chemins_test["neg"], label=0)

# Combiner en un seul DataFrame
df_test = pd.concat([df_test_pos, df_test_neg], ignore_index=True)

# Mélanger les lignes
df_test = df_test.sample(frac=1, random_state=42).reset_index(drop=True)

# Aperçu
df_test.head()


,texte,label
0,"I don't cry easily over movies, but I have to ...",1
1,This movie stinks. IMDb needs negative numbers...,0
2,"I loved the film ""Eddie Monroe"". The film had ...",1
3,This movie made my face hurt. I don't understa...,0
4,I think Downey was perhaps inspired by French ...,0


In [6]:
output_path_train = "data/df_train.parquet"
output_path_test = "data/df_test.parquet"

df_train.to_parquet(output_path_train, index=False)
df_test.to_parquet(output_path_test, index=False)